In [88]:
import pandas as pd
import numpy as np
from io import StringIO
from datetime import date

import requests

### Сначала определим год, за какой будем смотреть данные.

In [4]:
login = 'p-syrnev'

In [7]:
target_year = 1994 + hash(f'{login}') % 23

In [9]:
print(f'будем смотреть данные за {target_year}г.')

будем смотреть данные за 2002u


### Task 1, load data

In [15]:
df = pd.read_csv('vgsales.csv')

In [16]:
df.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16598 entries, 0 to 16597
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Rank          16598 non-null  int64  
 1   Name          16598 non-null  object 
 2   Platform      16598 non-null  object 
 3   Year          16327 non-null  float64
 4   Genre         16598 non-null  object 
 5   Publisher     16540 non-null  object 
 6   NA_Sales      16598 non-null  float64
 7   EU_Sales      16598 non-null  float64
 8   JP_Sales      16598 non-null  float64
 9   Other_Sales   16598 non-null  float64
 10  Global_Sales  16598 non-null  float64
dtypes: float64(6), int64(1), object(4)
memory usage: 1.4+ MB


In [24]:
df = df.query('Year == @target_year')

df.Year = df.Year.astype('int')

In [25]:
df.Year.unique()

array([2002])

In [26]:
# @task()
def get_target_year_data():
    # загрущим и отфильтруем датасет по заданному году
    login = 'p-syrnev'
    target_year = 1994 + hash(f'{login}') % 23
    path = 'vgsales.csv'
    # path = '/var/lib/airflow/airflow.git/dags/a.batalov/vgsales.csv'

    print(f'Будем смотреть данные за {target_year}г.')
    df_target_year = pd.read_csv(path).query('Year == @target_year')
    df_target_year.Year = df_target_year.Year.astype('int') 
    return df_target_year.to_csv(index=False)
    

### Task 2
Какая игра была самой продаваемой в этом году во всем мире?

In [32]:
df.groupby(by='Name', as_index=False) \
    .agg({'Global_Sales':'sum'}) \
    .rename(columns={'Global_Sales':'Total_Sales'}) \
    .sort_values(by='Total_Sales', ascending=False) \
    .iloc[0].Name

'Grand Theft Auto: Vice City'

In [34]:
# @task()
def get_best_selling_game(target_year_data : str):
    df = pd.read_csv(StringIO(target_year_data))

    return df.groupby(by='Name', as_index=False) \
        .agg({'Global_Sales':'sum'}) \
        .rename(columns={'Global_Sales':'Total_Sales'}) \
        .sort_values(by='Total_Sales', ascending=False) \
        .iloc[0].Name

### Task 3
Игры какого жанра были самыми продаваемыми в Европе? Перечислить все, если их несколько

In [56]:
df_genre_eu_sales = df.groupby(by='Genre', as_index=False) \
    .agg({'EU_Sales':'sum'})

max_sales = df_genre_eu_sales.EU_Sales.max()

','.join(df_genre_eu_sales.query('EU_Sales == @max_sales').Genre.tolist())


'Action'

In [57]:
#@task()
def get_best_selling_games_eu(target_year_data : str):
    df = pd.read_csv(StringIO(target_year_data))

    df_genre_eu_sales = df.groupby(by='Genre', as_index=False).agg({'EU_Sales':'sum'})

    max_sales = df_genre_eu_sales.EU_Sales.max()

    return ', '.join(df_genre_eu_sales.query('EU_Sales == @max_sales').Genre.tolist())

### Task 4
На какой платформе было больше всего игр, которые продались более чем миллионным тиражом в Северной Америке?
Перечислить все, если их несколько

In [67]:
df['IsGreatThan1M'] = df.NA_Sales > 1.0

df_platform_count = df.groupby(by=['Platform'], as_index=False) \
    .agg({'IsGreatThan1M':'sum'}) \
    .rename(columns={'IsGreatThan1M':'Count_games'})

max_1m = df_platform_count.Count_games.max()

', '.join(df_platform_count.query('Count_games == @max_1m').Platform.tolist())

'PS2'

In [69]:
# @task()
def get_1m_platform_na(target_year_data : str):
    df = pd.read_csv(StringIO(target_year_data))

    df['IsGreatThan1M'] = df.NA_Sales > 1.0

    df = df.groupby(by=['Platform'], as_index=False) \
        .agg({'IsGreatThan1M':'sum'}) \
        .rename(columns={'IsGreatThan1M':'Count_games'})

    max_1m = df.Count_games.max()

    return ', '.join(df.query('Count_games == @max_1m').Platform.tolist())


### Task 5
У какого издателя самые высокие средние продажи в Японии?
Перечислить все, если их несколько

In [74]:
df_avg_jp = df.groupby(by='Publisher', as_index=False) \
                .agg({'JP_Sales':'mean'})

max_avg_jp = df_avg_jp.JP_Sales.max()

', '.join(df_avg_jp.query('JP_Sales == @max_avg_jp').Publisher.tolist())

'Nintendo'

In [76]:
# @task()
def get_publisher_with_max_avegrage_jp(target_year_data : str):
    df = pd.read_csv(StringIO(target_year_data))

    df_avg_jp = df.groupby(by='Publisher', as_index=False) \
                .agg({'JP_Sales':'mean'})

    max_avg_jp = df_avg_jp.JP_Sales.max()

    return ', '.join(df_avg_jp.query('JP_Sales == @max_avg_jp').Publisher.tolist())

### Task 6
Сколько игр продались лучше в Европе, чем в Японии?

In [84]:
df_game_eu_vs_jp =df.groupby(by='Name') \
    .agg({'EU_Sales':'sum', 'JP_Sales' : 'sum'})

(df_game_eu_vs_jp.EU_Sales > df_game_eu_vs_jp.JP_Sales).sum()

484

In [85]:
# @task()
def get_count_eu_games(target_year_data : str):
    df = pd.read_csv(StringIO(target_year_data))

    df_game_eu_vs_jp =df.groupby(by='Name') \
        .agg({'EU_Sales':'sum', 'JP_Sales' : 'sum'})

    return (df_game_eu_vs_jp.EU_Sales > df_game_eu_vs_jp.JP_Sales).sum()


### Task 7
Вывод в консоль

In [49]:
date.today()

datetime.date(2022, 8, 22)

In [53]:
# замокаем метод для получения airflow контекста
def get_current_context():
    return {'ds':date.today()}

In [86]:
# @task()
def print_result(best_selling_game,
                 best_selling_games_eu,
                 platform_with_1m,
                 publisher_with_max_avegrage_jp,
                 eu_vs_jp):
    
    context = get_current_context()

    date = context['ds']
    print(f'''Дата: {date}

    1. Какая игра была самой продаваемой в этом году во всем мире?
    *{best_selling_game}*

    2. Игры какого жанра были самыми продаваемыми в Европе?
    {best_selling_games_eu}

    3. На какой платформе было больше всего игр, которые продались более чем миллионным тиражом в Северной Америке?
    {platform_with_1m}

    4. У какого издателя самые высокие средние продажи в Японии?
    {publisher_with_max_avegrage_jp}
    
    5. Сколько игр продались лучше в Европе, чем в Японии?
    {eu_vs_jp}
    ''')
    

### Test

In [87]:
target_year_data = get_target_year_data()
best_selling_game = get_best_selling_game(target_year_data)
best_selling_games_eu = get_best_selling_games_eu(target_year_data)
platform_with_1m = get_1m_platform_na(target_year_data)
publisher_with_max_avegrage_jp = get_publisher_with_max_avegrage_jp(target_year_data)
eu_vs_jp = get_count_eu_games(target_year_data)

print_result(best_selling_game, best_selling_games_eu, platform_with_1m, publisher_with_max_avegrage_jp, eu_vs_jp)

Будем смотреть данные за 2002г.
Дата: 2022-08-22

    1. Какая игра была самой продаваемой в этом году во всем мире?
    *Grand Theft Auto: Vice City*

    2. Игры какого жанра были самыми продаваемыми в Европе?
    Action

    3. На какой платформе было больше всего игр, которые продались более чем миллионным тиражом в Северной Америке?
    PS2

    4. У какого издателя самые высокие средние продажи в Японии?
    Nintendo
    
    5. Сколько игр продались лучше в Европе, чем в Японии?
    484
    


In [95]:
r = requests.get('https://api.telegram.org/bot5407072426:AAFCawYX4BjbHDTdbsO9gKDlCScW5cgJhSM/getUpdates')

In [96]:
r.content

b'{"ok":true,"result":[{"update_id":766037090,\n"message":{"message_id":2,"from":{"id":351301198,"is_bot":false,"first_name":"Pavel","last_name":"Syrnev","username":"psyrnev","language_code":"ru"},"chat":{"id":351301198,"first_name":"Pavel","last_name":"Syrnev","username":"psyrnev","type":"private"},"date":1661191276,"text":"\\u0442\\u0435\\u0441\\u0442\\u043e\\u0432\\u043e\\u0435"}}]}'